In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images\01.xml,163,324,199,264
1,./images\02.xml,1149,1716,577,734
2,./images\03.xml,91,820,120,380
3,./images\04.xml,112,981,217,396
4,./images\05.xml,162,576,197,377


In [3]:
import xml.etree.ElementTree as xet

In [4]:
filename = df['filepath'][0]
filename

'./images\\01.xml'

In [5]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./images',filename_image)
    return filepath_image

In [6]:
getFilename(filename)

'./images\\2A8CF8C600000578-0-image-a-86_1514912464951.jpg'

In [8]:
image_path = list(df['filepath'].apply(getFilename))
image_path

['./images\\2A8CF8C600000578-0-image-a-86_1514912464951.jpg',
 './images\\08a39e988b80453e921984574e3817d6.jpg',
 './images\\9a909c40785b46b2a06e27c398db41a3.jpg',
 './images\\9b138d6dc3d648a0b5778adb91b7cad2.JPG',
 './images\\12b199e5-e470-4c23-a46d-a9c3d3b31f86.jpg',
 './images\\37ced30df88e576cfbd04e9fef93882f.jpg',
 './images\\81f3c169bb37c500f07a8f9ac5087979.jpg',
 './images\\200.jpg',
 './images\\606-bentley-60el-front1-1024x600.jpg',
 './images\\2053a228-b548-4c27-97d4-f3f2771886f8.jpg',
 './images\\2586.jpg',
 './images\\36587192-0-image-a-24_1624525829907.jpg',
 './images\\252258583_370616978184783_6575372095692768733_n.jpg',
 './images\\20180402113123_NumberPlate_Swift.jpg',
 './images\\AP CM Jagan car.jpg',
 './images\\audi.jpg',
 './images\\car_number_plate_maker_1531338740_c28ed6f6.jpg',
 './images\\chrome-car-number-plate-500x500.jpg',
 './images\\dc2b4fd7-b397-49cd-a1b1-4e8db18d0a88.jpg',
 './images\\delhi number plate.jpg',
 './images\\dfaaf29f7dabbc05ae6d793b6c687106.j

In [9]:
####verify image and output####


In [21]:
file_path = image_path[0]
file_path

'./images\\2A8CF8C600000578-0-image-a-86_1514912464951.jpg'

In [22]:
img = cv2.imread(file_path)

cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
# 163	324	199	264
cv2.rectangle(img,(163,199),(324,264),(0,255,0),3)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
####Data Preprocessing####

In [25]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [26]:
labels = df.iloc[:,1:].values

In [27]:
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # normalization
    # normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # normalized output
    # -------------- append
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [28]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [29]:
X.shape,y.shape

((40, 224, 224, 3), (40, 4))

In [30]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((32, 224, 224, 3), (8, 224, 224, 3), (32, 4), (8, 4))

In [31]:
####Deep Learning Model####

In [32]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [33]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False,
                                     input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)
# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

219062272/219055592 [==============================] - 8s 0us/step


In [34]:
# complie model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

__________________________________________________________________________________________________
conv2d_129 (Conv2D)             (None, 12, 12, 128)  139264      block17_13_ac[0][0]              
__________________________________________________________________________________________________
batch_normalization_129 (BatchN (None, 12, 12, 128)  384         conv2d_129[0][0]                 
__________________________________________________________________________________________________
activation_129 (Activation)     (None, 12, 12, 128)  0           batch_normalization_129[0][0]    
__________________________________________________________________________________________________
conv2d_130 (Conv2D)             (None, 12, 12, 160)  143360      activation_129[0][0]             
__________________________________________________________________________________________________
batch_normalization_130 (BatchN (None, 12, 12, 160)  480         conv2d_130[0][0]                 
__________

In [35]:
####Model training####

In [36]:
from tensorflow.keras.callbacks import TensorBoard

In [37]:
tfb = TensorBoard('object_detection')

In [38]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/100
1/4 [======>.......................] - ETA: 0s - loss: 0.0794WARNING:tensorflow:From D:\anaconda\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
4/4 [==============================] - 8s 2s/step - loss: 0.0936 - val_loss: 0.0740
Epoch 2/100
4/4 [==============================] - 3s 815ms/step - loss: 0.0858 - val_loss: 0.1030
Epoch 3/100
4/4 [==============================] - 3s 766ms/step - loss: 0.0929 - val_loss: 0.0868
Epoch 4/100
4/4 [==============================] - 3s 782ms/step - loss: 0.0851 - val_loss: 0.0845
Epoch 5/100
4/4 [==============================] - 3s 807ms/step - loss: 0.0863 - val_loss: 0.0984
Epoch 6/100
4/4 [==============================] - 3s 708ms/step - loss: 0.0942 - val_loss: 0.0892
Epoch 7/100
4/4 [==============================] - 3s 692ms/step - loss: 0.0835

Epoch 80/100
4/4 [==============================] - 3s 791ms/step - loss: 0.0578 - val_loss: 0.0620
Epoch 81/100
4/4 [==============================] - 3s 707ms/step - loss: 0.0584 - val_loss: 0.0600
Epoch 82/100
4/4 [==============================] - 3s 767ms/step - loss: 0.0587 - val_loss: 0.0600
Epoch 83/100
4/4 [==============================] - 3s 749ms/step - loss: 0.0600 - val_loss: 0.0599
Epoch 84/100
4/4 [==============================] - 3s 745ms/step - loss: 0.0593 - val_loss: 0.0590
Epoch 85/100
4/4 [==============================] - 3s 676ms/step - loss: 0.0572 - val_loss: 0.0591
Epoch 86/100
4/4 [==============================] - 3s 768ms/step - loss: 0.0568 - val_loss: 0.0591
Epoch 87/100
4/4 [==============================] - 3s 631ms/step - loss: 0.0568 - val_loss: 0.0608
Epoch 88/100
4/4 [==============================] - 3s 631ms/step - loss: 0.0574 - val_loss: 0.0596
Epoch 89/100
4/4 [==============================] - 3s 654ms/step - loss: 0.0566 - val_loss: 0.0587


In [39]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=200,
                    validation_data=(x_test,y_test),callbacks=[tfb],initial_epoch=101)

Epoch 102/200
4/4 [==============================] - 3s 846ms/step - loss: 0.0562 - val_loss: 0.0586
Epoch 103/200
4/4 [==============================] - 3s 684ms/step - loss: 0.0562 - val_loss: 0.0588
Epoch 104/200
4/4 [==============================] - 3s 665ms/step - loss: 0.0560 - val_loss: 0.0588
Epoch 105/200
4/4 [==============================] - 3s 692ms/step - loss: 0.0563 - val_loss: 0.0589
Epoch 106/200
4/4 [==============================] - 3s 763ms/step - loss: 0.0562 - val_loss: 0.0587
Epoch 107/200
4/4 [==============================] - 3s 789ms/step - loss: 0.0555 - val_loss: 0.0589
Epoch 108/200
4/4 [==============================] - 3s 733ms/step - loss: 0.0550 - val_loss: 0.0585
Epoch 109/200
4/4 [==============================] - 3s 783ms/step - loss: 0.0542 - val_loss: 0.0589
Epoch 110/200
4/4 [==============================] - 3s 748ms/step - loss: 0.0548 - val_loss: 0.0587
Epoch 111/200
4/4 [==============================] - 3s 679ms/step - loss: 0.0544 - val_los

4/4 [==============================] - 3s 651ms/step - loss: 0.0172 - val_loss: 0.0108
Epoch 184/200
4/4 [==============================] - 3s 636ms/step - loss: 0.0156 - val_loss: 0.0114
Epoch 185/200
4/4 [==============================] - 3s 641ms/step - loss: 0.0109 - val_loss: 0.0084
Epoch 186/200
4/4 [==============================] - 3s 634ms/step - loss: 0.0115 - val_loss: 0.0102
Epoch 187/200
4/4 [==============================] - 3s 636ms/step - loss: 0.0119 - val_loss: 0.0190
Epoch 188/200
4/4 [==============================] - 3s 657ms/step - loss: 0.0116 - val_loss: 0.0092
Epoch 189/200
4/4 [==============================] - 3s 631ms/step - loss: 0.0082 - val_loss: 0.0249
Epoch 190/200
4/4 [==============================] - 3s 650ms/step - loss: 0.0152 - val_loss: 0.0159
Epoch 191/200
4/4 [==============================] - 3s 631ms/step - loss: 0.0140 - val_loss: 0.0074
Epoch 192/200
4/4 [==============================] - 3s 632ms/step - loss: 0.0099 - val_loss: 0.0192
Epoc

In [41]:
model.save('./models/object_detection.h5')